In [2]:
#Libs
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as Pd

In [3]:
#sparkSession
spark = SparkSession.builder.appName("Sessao").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/02 14:13:02 WARN Utils: Your hostname, DESKTOP-FBCHK7G, resolves to a loopback address: 127.0.1.1; using 172.19.227.193 instead (on interface eth0)
26/02/02 14:13:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 14:13:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#open first file
df_main = spark.read.option("header","True").parquet(f"../data/bronze/bronze_amazon_sales.parquet")

In [5]:
#check schema
df_main.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: string (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- promotion-ids: string (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- fulfilled-by: string (nullable = true)



In [6]:
#check number of registers
df_main.count()

120378

In [7]:
df_main.show(10)

+------+-------------------+--------+---------+----------+--------------+------------------+-------+------------------+--------+----+----------+--------------+---+--------+------+----------------+-------------+----------------+------------+--------------------+-----+------------+
| index|           Order ID|    Date|   Status|Fulfilment|Sales Channel |ship-service-level|  Style|               SKU|Category|Size|      ASIN|Courier Status|Qty|currency|Amount|       ship-city|   ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|
+------+-------------------+--------+---------+----------+--------------+------------------+-------+------------------+--------+----+----------+--------------+---+--------+------+----------------+-------------+----------------+------------+--------------------+-----+------------+
|119095|171-0000547-8192359|06-07-22|  Shipped|    Amazon|     Amazon.in|         Expedited|JNE2032| JNE2032-KR-205-XL|   kurta|  XL|B0768J7VQ1|       Shippe

##General check in database

In [8]:
lst = [df_main]

for df in lst:
    cols = df.columns

    df_null = df.select([
        F.sum(F.col(c).isNull().cast("integer")).alias(c)
        for c in cols
    ]).show()

+-----+--------+----+------+----------+--------------+------------------+-----+---+--------+----+----+--------------+---+--------+------+---------+----------+----------------+------------+-------------+---+------------+
|index|Order ID|Date|Status|Fulfilment|Sales Channel |ship-service-level|Style|SKU|Category|Size|ASIN|Courier Status|Qty|currency|Amount|ship-city|ship-state|ship-postal-code|ship-country|promotion-ids|B2B|fulfilled-by|
+-----+--------+----+------+----------+--------------+------------------+-----+---+--------+----+----+--------------+---+--------+------+---------+----------+----------------+------------+-------------+---+------------+
|    0|       0|   0|     0|         0|             0|                 0|    0|  0|       0|   0|   0|          6387|  0|    7401|  7401|       28|        28|              28|          28|        46927|  0|       84002|
+-----+--------+----+------+----------+--------------+------------------+-----+---+--------+----+----+--------------+---

In [9]:
df_analysis = df.select(["Courier Status", "currency","Amount","ship-city","ship-state","ship-country","promotion-ids","fulfilled-by"])

In [11]:
df_analysis_nulls = df_analysis.filter(
    F.col("Courier Status").isNull() |
    F.col("currency").isNull() |
    F.col("Amount").isNull()
)

In [12]:
df_analysis_nulls.show(20)

+--------------+--------+------+--------------------+----------------+------------+-------------+------------+
|Courier Status|currency|Amount|           ship-city|      ship-state|ship-country|promotion-ids|fulfilled-by|
+--------------+--------+------+--------------------+----------------+------------+-------------+------------+
|     Cancelled|    NULL|  NULL|     THANE(W), thane|     MAHARASHTRA|          IN|         NULL|        NULL|
|          NULL|     INR|708.57|              KALWAN|     MAHARASHTRA|          IN|         NULL|   Easy Ship|
|     Cancelled|    NULL|  NULL|             KURNOOL|  ANDHRA PRADESH|          IN|         NULL|        NULL|
|     Cancelled|    NULL|  NULL|             NALBARI|           ASSAM|          IN|         NULL|        NULL|
|          NULL|     INR|336.19|             KOLKATA|     WEST BENGAL|          IN|         NULL|   Easy Ship|
|          NULL|    NULL|  NULL|            VARANASI|   UTTAR PRADESH|          IN|         NULL|   Easy Ship|
|